In [204]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

In [205]:
def box_area(box):
    return max(0, box[2] - box[0]) * max(0, box[3] - box[1])

def intersection_area(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    return interW * interH

In [206]:
# Load YOLO model
model = YOLO(r"C:\Users\youssef\Downloads\YOLOV8model_MUCH trained.pt")

# Load input video
cap = cv2.VideoCapture(r"C:\Users\youssef\Downloads\6474370-uhd_3840_2160_25fps.mp4")

In [207]:
# Output video setup
output_file = "output/output_video.avi"
os.makedirs("output", exist_ok=True)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output size (resize to 1020x600)
out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), fps, (800, 600))

# Font for text on frame
font = cv2.FONT_HERSHEY_SIMPLEX

frame_idx = 0

In [208]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (800, 600))

    # Detect safety helmets, vests, and heads
    results = model(frame)
    result = results[0]  # first (and only) detection result
    pred = result.boxes.data.cpu().numpy()  # [x1, y1, x2, y2, confidence, class]

    helmets = pred[pred[:, 5] == 0]
    vests = pred[pred[:, 5] == 1]
    heads = pred[pred[:, 5] == 2]

    # Initialize warning messages list
    warning_messages = []

    # Draw bounding boxes and labels for helmets
    for helmet in helmets:
        x1, y1, x2, y2 = map(int, helmet[:4])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, 'Helmet', (x2 + 5, y1 + 15), font, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
        
        # If helmet found but no vest
        if len(vests) == 0:
            warning_text = "WARNING:  No Vest"
            warning_messages.append(warning_text)
            print(f"Frame {frame_idx}: {warning_text}")

    # Draw bounding boxes and labels for vests
    for vest in vests:
        x1, y1, x2, y2 = map(int, vest[:4])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, 'Vest', (x1, y1 - 10), font, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
        
        # If vest found but no helmet
        if len(helmets) == 0:
            warning_text = "WARNING:  No Helmet"
            warning_messages.append(warning_text)
            print(f"Frame {frame_idx}: {warning_text}")

    # Check heads for missing gear
    for head_idx, head in enumerate(heads):
        x1, y1, x2, y2 = map(int, head[:4])
        has_helmet = False
        has_vest = False

        # Check helmet overlap
        for helmet in helmets:
            inter_area = intersection_area([x1, y1, x2, y2], helmet[:4])
            if inter_area > 0.2 * box_area([x1, y1, x2, y2]):  # >20% overlap
                has_helmet = True
                break

        # Check vest overlap
        for vest in vests:
            inter_area = intersection_area([x1, y1, x2, y2], vest[:4])
            if inter_area > 0.2 * box_area([x1, y1, x2, y2]):  # >20% overlap
                has_vest = True
                break

        # If head is found but no vest
        if not has_vest:
            warning_text = "WARNING:  No Vest No Helmet"
            warning_messages.append(warning_text)
            print(f"Frame {frame_idx}, Head {head_idx}: {warning_text}")

    # Draw bounding boxes and labels for heads
    for head in heads:
        x1, y1, x2, y2 = map(int, head[:4])
        if x1 <= x2 and y1 <= y2:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 100, 255), 2)
            cv2.rectangle(frame, (x1, y1 - 20), (x2, y1), (245, 222, 179), cv2.FILLED)
            cv2.putText(frame, 'Head', (x1, y1 - 10), font, 0.5, (255, 0, 255), 2, cv2.LINE_AA)

    # Display warnings at the bottom left
    y_position = frame.shape[0] - 30  # Start from bottom
    for warning in warning_messages:
        text_size = cv2.getTextSize(warning, font, 0.5, 2)[0]
        # Add black background for better visibility
        cv2.rectangle(frame,
                     (10, y_position - text_size[1]),
                     (10 + text_size[0], y_position + 5),
                     (0, 0, 0), -1)
        cv2.putText(frame, warning, (10, y_position), font, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
        y_position -= 30  # Move up for next warning

    # Write the frame to the output video
    out.write(frame)

    # Show the frame
    cv2.imshow('frame', frame)

    # Quit if user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_idx += 1

cap.release()
out.release()
cv2.destroyAllWindows()

print("Processing completed successfully. Output file saved as", output_file)

0: 480x640 1 Vest, 1 Head, 887.7ms
Speed: 8.4ms preprocess, 887.7ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Vest, 1 Head, 998.9ms
Speed: 6.1ms preprocess, 998.9ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Vest, 1 Head, 847.0ms
Speed: 5.6ms preprocess, 847.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Vest, 1 Head, 833.7ms
Speed: 7.3ms preprocess, 833.7ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Vest, 1 Head, 863.6ms
Speed: 5.2ms preprocess, 863.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Vest, 1 Head, 826.0ms
Speed: 5.0ms preprocess, 826.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Vest, 1 Head, 823.8ms
Speed: 5.6ms preprocess, 823.8ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Vest, 811.9ms
Speed: 4.6ms preprocess, 811.9ms inference